> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.2`: Model building demo
For many applications of `REINVENT`, we already have some prior knowledge on a project that we would like to incorporate. One example of how this can be achieved are *predictive models* trained on a collection of compounds, for example `QSAR` models that relate the structure of a compound to an activity / potency endpoint. In this demo, we will explain how to build `scikit-learn` models that are compatible with `REINVENT`. Note, that our model will have the ending `.pkl`, as we need to save the model's parameters in a serialized fashion ("[pickled](https://docs.python.org/3/library/pickle.html)").

Our input dataset will be `DRD2` (see also the "complete use-case notebook"), for which we have compiled two `CSV` files with the compounds' `SMILES` in one column and a read-out value in another (see below). We will train a `Random Forest` regressor here, but you can choose any backend algorithm that provides a `scikit-learn` interface. As of now, `REINVENT` supports 4 different kinds of molecular fingerprints and we will need to calculate them before we can train our model.

The following imports are required to execute the code. Please update the output directory path if you want to store the temporary files and the final model in another place.

In [1]:
import os
import pandas as pd
import numpy as np
import random
import pickle
import sklearn.ensemble
from sklearn.metrics import roc_auc_score, mean_squared_error

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys, PandasTools
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.Draw import IPythonConsole
from IPython.core.display import display, HTML

from shutil import copyfile

import matplotlib.pyplot as plt
import seaborn as sns

# set plotting parameters
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

In [2]:
# --------- change these path variables as required
output_dir = os.path.expanduser("~/Desktop/REINVENT_model_building_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

# copy data sets to temporary folder for inspection
copyfile(os.path.join(ipynb_path, "data/drd2.train.csv"),
         os.path.join(output_dir, "drd2.train.csv"))
copyfile(os.path.join(ipynb_path, "data/drd2.test.csv"),
         os.path.join(output_dir, "drd2.test.csv"));

## Data
Before proceeding, let us inspect the datasets. We have split them into a training and test sets, each of which has `canonical` (the `SMILES`) and `activity` (either `1` or `0`, corresponding to `active` and `inactive` respectively) columns. This means, we have a (binary) classification problem.

In [3]:
train = pd.read_csv(os.path.join(output_dir, "drd2.train.csv"))
test = pd.read_csv(os.path.join(output_dir, "drd2.test.csv"))


print("# obs in train: ", train.shape[0])
print("# obs in test: ", test.shape[0])
train.head()

# obs in train:  275768
# obs in test:  68944


,canonical,activity
0,COc1ccc(NC(=O)CC2C(=O)N(c3ccc(Cl)cc3)C(=S)N2CC...,0
1,O=C(CSc1oc(-c2ccccc2)nc1S(=O)(=O)c1ccc(Br)cc1)...,0
2,Cn1c(=S)n(CCC(=O)O)c2ccccc21,0
3,Clc1ccc(NN=Cc2ccncc2)cc1Cl,0
4,O=C(C=NO)NCCCNCc1ccccc1,0


## Descriptors
As mentioned above, we need to calculate descriptors (fingerprints) which are understood by `REINVENT`. Below is some code to facilitate this for one example configuration (`ECFP6` with counts), but feel free to adapt it if you feel your model could benefit from it.

In [4]:
def smiles_to_mols(query_smiles):
    mols = [Chem.MolFromSmiles(smile) for smile in query_smiles]
    valid = [0 if mol is None else 1 for mol in mols]
    valid_idxs = [idx for idx, boolean in enumerate(valid) if boolean == 1]
    valid_mols = [mols[idx] for idx in valid_idxs]
    return valid_mols, valid_idxs


class Descriptors:

    def __init__(self, data):
        self._data = data

    def ECFP(self, radius, nBits):
        fingerprints = []
        mols, idx = smiles_to_mols(self._data)
        fp_bits = [AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) for mol in mols]
        for fp in fp_bits:
            fp_np = np.zeros((1, nBits), dtype=np.int32)
            DataStructs.ConvertToNumpyArray(fp, fp_np)
            fingerprints.append(fp_np)
        return fingerprints, idx

    def ECFP_counts(self, radius, useFeatures, useCounts=True):
        mols, valid_idx = smiles_to_mols(self._data)
        fps = [AllChem.GetMorganFingerprint(mol, radius, useCounts=useCounts, useFeatures=useFeatures) for mol in mols]
        size = 2048
        nfp = np.zeros((len(fps), size), np.int32)
        for i, fp in enumerate(fps):
            for idx, v in fp.GetNonzeroElements().items():
                nidx = idx % size
                nfp[i, nidx] += int(v)
        return nfp, valid_idx

    def Avalon(self, nBits):
        mols, valid_idx = smiles_to_mols(self._data)
        fingerprints = []
        fps = [pyAvalonTools.GetAvalonFP(mol, nBits=nBits) for mol in mols]
        for fp in fps:
            fp_np = np.zeros((1, nBits), dtype=np.int32)
            DataStructs.ConvertToNumpyArray(fp, fp_np)
            fingerprints.append(fp_np)
        return fingerprints, valid_idx

    def MACCS_keys(self):
        mols, valid_idx = smiles_to_mols(self._data)
        fingerprints = []
        fps = [MACCSkeys.GenMACCSKeys(mol) for mol in mols]
        for fp in fps:
            fp_np = np.zeros((1, ), dtype=np.int32)
            DataStructs.ConvertToNumpyArray(fp, fp_np)
            fingerprints.append(fp_np)
        return fingerprints, valid_idx

def get_ECFP6_counts(inp):
    if not isinstance(inp, list):
        inp = list(inp)
    desc = Descriptors(inp)
    fps, _ = desc.ECFP_counts(radius=3, useFeatures=True, useCounts=True)
    return fps

In [5]:
train_fps = get_ECFP6_counts(train["canonical"])
test_fps = get_ECFP6_counts(test["canonical"])

print(train_fps)

[[ 9  1  4 ...  0  0  0]
 [ 9  1  4 ...  0  0  0]
 [ 3  1  2 ...  0  0  0]
 ...
 [11  0  1 ...  1  0  0]
 [11  1  1 ...  0  0  0]
 [ 6  0  2 ...  0  0  0]]


## Model training
To train your model (i.e. fit the model's parameters to the training data), execute the following cell. Note, that `REINVENT` will access the `proba` property of the model to get a probability rather than a predicted label. If you want to optimize the hyper-parameters of you model, we suggest you use a cross-validation approach (we aim to publish our in-house method based on [optuna](https://optuna.org) soon).

In [6]:
# initialize a "Random Forest Classifier" (choice of hyper-parameters somewhat arbitrary)
RFclassifier = sklearn.ensemble.RandomForestClassifier(max_depth=20,
                                                       max_features="auto",
                                                       n_estimators=100,
                                                       class_weight="balanced")

# fit to training data
RFclassifier.fit(train_fps, train["activity"])
y_pred = RFclassifier.predict(X=train_fps)
train_score = roc_auc_score(y_true=train["activity"], y_score=y_pred)
print(train_score)

0.987600796694758


## Evaluate performance on test set
To get a better idea on how well (or meager) our model performs, we can evaluate it on the test (hold-out) set. These observations have not been used for fitting the parameters and are thus a good proxy for how the model would fare for new compounds generated, e.g. by `REINVENT`. Note, that we do not strive to give a comprehensive tutorial how to build good, well generalizing models in this demonstration and there are quite a few important considerations to contemplate before actually using a model in production.

In [7]:
y_pred_test = RFclassifier.predict(X=test_fps)
test_score = roc_auc_score(y_true=test["activity"], y_score=y_pred_test)
print(test_score)

0.961789032126087


## Write out final model
Finally, we will write-out the model. Usually, we will build it using all the observations available. While this maximizes the amount of knowledge available to the model, it is noteworthy that this comes at the trade-off of not being able to independently assess the model's performance anymore.

In [8]:
RFclassifier_final = sklearn.ensemble.RandomForestClassifier(max_depth=20,
                                                             max_features="auto",
                                                             n_estimators=100,
                                                             class_weight="balanced")

# fit to all data points
complete_fps = np.concatenate((train_fps, test_fps), axis=0)
complete_y = pd.concat((train["activity"], test["activity"]))
RFclassifier_final.fit(complete_fps, complete_y)
y_pred_final = RFclassifier.predict(X=complete_fps)
train_score = roc_auc_score(y_true=complete_y, y_score=y_pred_final)
print(train_score)

0.9824336871329722


In [9]:
# save the model
with open(os.path.join(output_dir, "DRD2_final_model.pkl"), "wb") as f:
    pickle.dump(RFclassifier_final, f)

## Integration into `REINVENT`
In order to use your new model as a component in the scoring function of `REINVENT`, you need to include a block with the appropriate parameter settings (see below). Note, that the descriptor definition needs to match.

```
{
  "component_type": "predictive_property",
  "name": "DRD2_pred_activity",
  "weight": 1,
  "specific_parameters": {
    "model_path": "/path/to/model/folder/DRD2_final_model.pkl",
    "scikit": "classification",
    "transformation: {
        "transformation_type": "no_transformation"
    },
    "descriptor_type": "ecfp_counts",
    "size": 2048,
    "radius": 3,
    "use_counts": True,
    "use_features": True
  }
}
```